In [ ]:
!pip install scikit-learn
!pip install pysindy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 10.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pysindy as ps

df = pd.read_excel('Archivo.xlsx')

# Ver las primeras filas del DataFrame
data_array = pd.DataFrame(df.values)

# Extraer X y t desde el archivo cargado (ajusta los nombres de columna según el archivo)
X = df['y'].values.reshape(-1, 1)
t = df['x'].values
X_extended = np.hstack([X, t.reshape(-1, 1)])

t, unique_indices = np.unique(t, return_index=True)

X = X[unique_indices]
#FUNCIONES
###########################################################
##                       FUNCIONES                       ##
###########################################################
# Crear las bibliotecas de funciones
polynomial_library = ps.PolynomialLibrary(degree=4)
fourier_library = ps.FourierLibrary()
tensor1 = polynomial_library*fourier_library
exp = [lambda x: 1.0 / (x+100), lambda x: np.exp(x)]
exp2 = [lambda x: '1.0 / (' + x + '100',
        lambda x: 'exp(-' + x + ')']
log = [lambda x: 1.0 / (x+100), lambda x: np.log(x)]
log2 = [lambda x: '1.0 / (' + x + '100',
        lambda x: 'log(-' + x + ')']
custom_exp = ps.CustomLibrary(library_functions=exp,
                          function_names=exp2)
customlog = ps.CustomLibrary(library_functions=log,
                          function_names=log2)
tensor2 = polynomial_library*custom_exp
tensor3 = custom_exp*fourier_library
tensor4 = polynomial_library*customlog
tensor5 = customlog*fourier_library
tensor6 = custom_exp*customlog
# Combinación de todas las librerias creadas
custom = ps.GeneralizedLibrary([tensor1,
                                tensor2,
                                tensor3,
                                tensor4,
                                tensor5,
                                tensor6,
                                custom_exp,
                                customlog])
# Combinar las bibliotecas de funciones
combined_library = ps.GeneralizedLibrary([
    polynomial_library,
    #fourier_library,
    #tensor1
    #custom
    #tensor1
])
##########################################################
##              Configurar el modelo SINDy              ##
##########################################################
differentiation_method = ps.FiniteDifference()
feature_library = combined_library
optimizer = ps.STLSQ(threshold=0.01)

model = ps.SINDy(
    differentiation_method=differentiation_method,
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=['y','t'],
)

# Ajustar el modelo y mostrar la ecuación
model.fit(X_extended, t=t)
model.print()

(y)' = 8.165 1 + -0.121 y + -0.027 t
(t)' = 1.024 1
